# 02. Tiền Xử Lý và Thiết Kế Đặc Trưng
Notebook này thực hiện tiền xử lý dữ liệu và thiết kế đặc trưng cho bài toán dự đoán kết quả học tập của sinh viên.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from scipy import stats
from src.data.loader import DataLoader
from src.data.cleaner import DataCleaner

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
%matplotlib inline

## 1. Tải Dữ Liệu

In [ ]:
loader = DataLoader()
df = loader.load_combined_data(merge=False)
print(f"Kích thước dữ liệu ban đầu: {df.shape}")

# Hiển thị thông tin cơ bản trước tiền xử lý
print("\nThông tin dữ liệu trước tiền xử lý:")
print(f"- Tổng số mẫu: {len(df)}")
print(f"- Tổng số đặc trưng: {len(df.columns)}")
print(f"- Số lượng giá trị thiếu: {df.isnull().sum().sum()}")
print(f"- Số lượng mẫu trùng lặp: {df.duplicated().sum()}")

## 2. Thống kê Trước Tiền Xử Lý

In [ ]:
# Thống kê trước tiền xử lý
pre_stats = {
    'Tổng số mẫu': len(df),
    'Tổng số đặc trưng': len(df.columns),
    'Giá trị thiếu': df.isnull().sum().sum(),
    'Mẫu trùng lặp': df.duplicated().sum(),
    'Số đặc trưng định danh': len(df.select_dtypes(include=['object']).columns),
    'Số đặc trưng số': len(df.select_dtypes(include=['int64', 'float64']).columns),
    'G3 - Trung vị': df['G3'].median(),
    'G3 - Độ lệch chuẩn': df['G3'].std()
}

print("=== THỐNG KÊ TRƯỚC TIỀN XỬ LÝ ===")
for key, value in pre_stats.items():
    print(f"{key}: {value}")

## 3. Phát Hiện và Xử Lý Ngoại Lai

In [ ]:
# Phát hiện ngoại lai sử dụng IQR Method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

# Phát hiện ngoại lai cho các biến số
numeric_columns = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']

print("Phát hiện ngoại lai bằng phương pháp IQR:")
outlier_counts = {}
for col in numeric_columns:
    if col in df.columns:
        outliers = detect_outliers_iqr(df, col)
        outlier_counts[col] = len(outliers)
        print(f"{col}: {len(outliers)} ngoại lai ({len(outliers)/len(df)*100:.2f}%)")

# Trực quan hóa một số biến có nhiều ngoại lai
high_outlier_vars = [var for var, count in outlier_counts.items() if count > 0]
if high_outlier_vars:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()
    
    for i, col in enumerate(high_outlier_vars[:6]):  # Hiển thị tối đa 6 biến
        df.boxplot(column=col, ax=axes[i])
        axes[i].set_title(f'Boxplot của {col}')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Xử lý ngoại lai - áp dụng Winsorization cho một số biến
from scipy.stats.mstats import winsorize

# Tạo bản sao để xử lý ngoại lai
df_processed = df.copy()

# Áp dụng Winsorization cho các biến có nhiều ngoại lai
variables_to_winsorize = ['absences', 'G1', 'G2', 'G3']  # Những biến có thể có ngoại lai

for var in variables_to_winsorize:
    if var in df_processed.columns:
        # Winsorize 5% giá trị cao nhất và thấp nhất
        df_processed[var] = winsorize(df_processed[var], limits=[0.05, 0.05])

print("Đã áp dụng Winsorization cho các biến có ngoại lai")

## 4. Tiền Xử Lý Chính

In [ ]:
# Khởi tạo DataCleaner
cleaner = DataCleaner()

# Loại bỏ các mẫu trùng lặp
initial_len = len(df_processed)
df_processed = df_processed.drop_duplicates()
print(f"Loại bỏ {initial_len - len(df_processed)} mẫu trùng lặp")

# Xử lý giá trị thiếu
missing_before = df_processed.isnull().sum().sum()
df_processed = df_processed.fillna(df_processed.median(numeric_only=True))  # Dùng median cho các cột số

# Đối với các cột phân loại, dùng giá trị phổ biến nhất
for col in df_processed.select_dtypes(include=['object']).columns:
    if df_processed[col].isnull().any():
        df_processed[col] = df_processed[col].fillna(df_processed[col].mode()[0] if not df_processed[col].mode().empty else 'Unknown')

missing_after = df_processed.isnull().sum().sum()
print(f"Xử lý giá trị thiếu: {missing_before} -> {missing_after}")

In [ ]:
# Mã hóa các biến phân loại
label_encoders = {}
categorical_columns = df_processed.select_dtypes(include=['object']).columns.tolist()

for col in categorical_columns:
    if col not in ['pass_fail']:  # Giữ nguyên nếu có cột pass_fail
        le = LabelEncoder()
        df_processed[col] = le.fit_transform(df_processed[col].astype(str))
        label_encoders[col] = le

print(f"Đã mã hóa {len(categorical_columns)} cột phân loại")

## 5. Thống kê Sau Tiền Xử Lý

In [ ]:
# Thống kê sau tiền xử lý
post_stats = {
    'Tổng số mẫu': len(df_processed),
    'Tổng số đặc trưng': len(df_processed.columns),
    'Giá trị thiếu': df_processed.isnull().sum().sum(),
    'Mẫu trùng lặp': df_processed.duplicated().sum(),
    'Số đặc trưng định danh': len(df_processed.select_dtypes(include=['object']).columns),
    'Số đặc trưng số': len(df_processed.select_dtypes(include=['int64', 'float64']).columns),
    'G3 - Trung vị': df_processed['G3'].median(),
    'G3 - Độ lệch chuẩn': df_processed['G3'].std()
}

print("=== THỐNG KÊ SAU TIỀN XỬ LÝ ===")
for key, value in post_stats.items():
    print(f"{key}: {value}")

In [ ]:
# Bảng so sánh trước - sau tiền xử lý
comparison_df = pd.DataFrame({
    'Trước tiền xử lý': [pre_stats[key] for key in pre_stats.keys()],
    'Sau tiền xử lý': [post_stats[key] for key in pre_stats.keys()]
}, index=pre_stats.keys())

print("\n=== BẢNG SO SÁNH TRƯỚC - SAU TIỀN XỬ LÝ ===")
print(comparison_df)

# Trực quan hóa sự thay đổi
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# So sánh số lượng mẫu
axes[0, 0].bar(['Trước', 'Sau'], [pre_stats['Tổng số mẫu'], post_stats['Tổng số mẫu']], color=['blue', 'green'])
axes[0, 0].set_title('So sánh số lượng mẫu')
axes[0, 0].set_ylabel('Số lượng')

# So sánh giá trị thiếu
axes[0, 1].bar(['Trước', 'Sau'], [pre_stats['Giá trị thiếu'], post_stats['Giá trị thiếu']], color=['red', 'green'])
axes[0, 1].set_title('So sánh giá trị thiếu')
axes[0, 1].set_ylabel('Số lượng')

# So sánh mẫu trùng lặp
axes[1, 0].bar(['Trước', 'Sau'], [pre_stats['Mẫu trùng lặp'], post_stats['Mẫu trùng lặp']], color=['red', 'green'])
axes[1, 0].set_title('So sánh mẫu trùng lặp')
axes[1, 0].set_ylabel('Số lượng')

# So sánh độ lệch chuẩn của G3
axes[1, 1].bar(['Trước', 'Sau'], [pre_stats['G3 - Độ lệch chuẩn'], post_stats['G3 - Độ lệch chuẩn']], color=['blue', 'green'])
axes[1, 1].set_title('So sánh độ lệch chuẩn G3')
axes[1, 1].set_ylabel('Độ lệch chuẩn')

plt.tight_layout()
plt.show()

## 6. Thiết Kế Đặc Trưng

In [ ]:
# Tạo các đặc trưng mới
df_features = df_processed.copy()

# Đặc trưng tương tác
df_features['study_failures_ratio'] = df_features['studytime'] / (df_features['failures'] + 1)  # +1 để tránh chia cho 0
df_features['medu_fedu_sum'] = df_features['Medu'] + df_features['Fedu']
df_features['alc_consumption'] = (df_features['Dalc'] + df_features['Walc']) / 2

# Đặc trưng nhị phân
df_features['high_absences'] = (df_features['absences'] > df_features['absences'].median()).astype(int)
df_features['high_studytime'] = (df_features['studytime'] > df_features['studytime'].median()).astype(int)

print(f"Số lượng đặc trưng sau khi tạo đặc trưng mới: {df_features.shape[1]}")
print(f"Đã tạo thêm {df_features.shape[1] - df_processed.shape[1]} đặc trưng mới")

## 7. Kiểm Soát Tham Số

In [ ]:
# Đặt seed cho tính tái lập
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Cấu hình tiền xử lý
PREPROCESS_CONFIG = {
    'winsorize_limits': [0.05, 0.05],
    'scaling_method': 'standard',
    'imputation_strategy': 'median',
    'random_state': RANDOM_STATE
}

print("Cấu hình tiền xử lý:")
for key, value in PREPROCESS_CONFIG.items():
    print(f"{key}: {value}")

## 8. Chuẩn Hóa Dữ Liệu

In [ ]:
# Chuẩn hóa các đặc trưng số
scaler = StandardScaler()
numeric_features = df_features.select_dtypes(include=[np.number]).columns.tolist()

# Loại trừ biến mục tiêu nếu nó là cột số
if 'G3' in numeric_features:
    numeric_features.remove('G3')
if 'pass_fail' in numeric_features:
    numeric_features.remove('pass_fail')

df_features[numeric_features] = scaler.fit_transform(df_features[numeric_features])

print(f"Đã chuẩn hóa {len(numeric_features)} đặc trưng số")
print(f"Kích thước dữ liệu cuối cùng: {df_features.shape}")

In [ ]:
# Lưu dữ liệu đã xử lý
output_path = '../data/processed/student_processed.csv'
df_features.to_csv(output_path, index=False)
print(f"Đã lưu dữ liệu đã xử lý vào: {output_path}")

## 9. Đánh giá & Kết Luận (tự động)

Phần này tóm tắt tác động của các bước tiền xử lý và thiết kế đặc trưng dựa trên các biến đã tính ở trên. Khi chạy lại notebook, nội dung sẽ tự cập nhật theo kết quả mới nhất.

In [ ]:
print("=== ĐÁNH GIÁ & KẾT LUẬN (TIỀN XỬ LÝ & ĐẶC TRƯNG) ===")

# 1) Trước–sau tiền xử lý
print("\n1) Thống kê trước–sau tiền xử lý (tóm tắt):")
print(f"   - Số mẫu: {pre_stats['Tổng số mẫu']:,} → {post_stats['Tổng số mẫu']:,}")
print(f"   - Missing values: {pre_stats['Giá trị thiếu']:,} → {post_stats['Giá trị thiếu']:,}")
print(f"   - Mẫu trùng lặp: {pre_stats['Mẫu trùng lặp']:,} → {post_stats['Mẫu trùng lặp']:,}")

removed_duplicates = int(pre_stats['Mẫu trùng lặp'] - post_stats['Mẫu trùng lặp'])
if removed_duplicates > 0:
    print(f"   - Đã loại bỏ khoảng {removed_duplicates:,} bản ghi trùng lặp")

# 2) Ngoại lai & winsorization
print("\n2) Ngoại lai và xử lý ngoại lai:")
if 'outlier_counts' in globals() and isinstance(outlier_counts, dict) and len(outlier_counts) > 0:
    top_outliers = sorted(outlier_counts.items(), key=lambda kv: kv[1], reverse=True)[:5]
    print("   - Top biến có nhiều ngoại lai (IQR):")
    for col, cnt in top_outliers:
        print(f"     + {col}: {cnt:,} ngoại lai ({cnt/len(df):.2%})")
else:
    print("   - Ghi chú: outlier_counts không khả dụng (có thể cell phát hiện ngoại lai chưa chạy)")

if 'variables_to_winsorize' in globals():
    print(f"   - Đã áp dụng Winsorization (limits={PREPROCESS_CONFIG.get('winsorize_limits', None)}) cho: {variables_to_winsorize}")

# 3) Mã hóa và chuẩn hóa
print("\n3) Mã hóa & chuẩn hóa:")
if 'categorical_columns' in globals():
    print(f"   - Số cột phân loại được mã hóa: {len(categorical_columns)}")
else:
    print("   - Số cột phân loại được mã hóa: (không xác định)")

if 'numeric_features' in globals():
    print(f"   - Số đặc trưng số được chuẩn hóa: {len(numeric_features)}")

# 4) Thiết kế đặc trưng
print("\n4) Thiết kế đặc trưng:")
if 'df_features' in globals() and 'df_processed' in globals():
    added = int(df_features.shape[1] - df_processed.shape[1])
    print(f"   - Số đặc trưng: {df_processed.shape[1]} → {df_features.shape[1]} (tạo thêm {added})")
    created_features = [
        'study_failures_ratio',
        'medu_fedu_sum',
        'alc_consumption',
        'high_absences',
        'high_studytime'
    ]
    available_created = [f for f in created_features if f in df_features.columns]
    if available_created:
        print(f"   - Các đặc trưng mới tiêu biểu: {available_created}")
else:
    print("   - Ghi chú: df_features/df_processed không khả dụng (có thể cell tạo đặc trưng chưa chạy)")

# 5) Đầu ra pipeline
print("\n5) Đầu ra pipeline:")
if 'output_path' in globals():
    print(f"   - Dữ liệu đã xử lý được lưu tại: {output_path}")
else:
    print("   - Dữ liệu đã xử lý được lưu tại: (không xác định đường dẫn)")

print("\n6) Kết luận:")
print("   - Sau tiền xử lý và thiết kế đặc trưng, dữ liệu sẵn sàng cho khai phá tri thức (clustering/rules) và mô hình hóa (classification)")
print("   - Các thống kê trước–sau cho phép kiểm chứng tác động của pipeline và hỗ trợ tính tái lập khi chạy lại")